# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

T. Pfeil  ->  T. Pfeil  |  ['T. Pfeil']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
Arxiv has 55 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.03816


extracting tarball to tmp_2409.03816...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:488: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

/tmp/ipykernel_2761/1211882699.py:51: LatexWarning: 2409.03829 did not run properly
bad escape \i at position 36
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2409.04046...

 done.
Retrieving document from  https://arxiv.org/e-print/2409.04405


extracting tarball to tmp_2409.04405...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.03816-b31b1b.svg)](https://arxiv.org/abs/2409.03816) | **TriPoD: Tri-Population size distributions for Dust evolution. Coagulation in vertically integrated hydrodynamic simulations of protoplanetary disks**  |
|| <mark>T. Pfeil</mark>, T. Birnstiel, <mark>H. Klahr</mark> |
|*Appeared on*| *2024-09-09*|
|*Comments*| *22 pages, 13 figures, accepted for publication in Astronomy and Astrophysics*|
|**Abstract**|            Context. Dust coagulation and fragmentation impact the structure and evolution of protoplanetary disks and set the initial conditions for planet formation. Dust grains dominate the opacities, they determine the cooling times of the gas, they influence the ionization state of the gas, and the grain surface area is an important parameter for the chemistry in protoplanetary disks. Therefore, dust evolution should not be ignored in numerical studies of protoplanetary disks. Available dust coagulation models are, however, too computationally expensive to be implemented in large-scale hydrodynamic simulations. This limits detailed numerical studies of protoplanetary disks, including these effects, mostly to one-dimensional models. Aims. We aim to develop a simple - yet accurate - dust coagulation model that can be implemented in hydrodynamic simulations of protoplanetary disks. Our model shall not significantly increase the computational cost of simulations and provide information about the local grain size distribution. Methods. The local dust size distributions are assumed to be truncated power laws. Such distributions can be characterized by two dust fluids (large and small grains) and a maximum particle size, truncating the power law. We compare our model to state-of-the-art dust coagulation simulations and calibrate it to achieve a good fit with these sophisticated numerical methods. Results. Running various parameter studies, we achieved a good fit between our simplified three-parameter model and DustPy, a state-of-the-art dust coagulation software. Conclusions. We present TriPoD, a sub-grid dust coagulation model for the PLUTO code. With TriPoD, we can perform two-dimensional, vertically integrated dust coagulation simulations on top of a hydrodynamic simulation. Studying the dust distributions in two-dimensional vortices and planet-disk systems is thus made possible.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.04046-b31b1b.svg)](https://arxiv.org/abs/2409.04046) | **Nonlinear dynamics of oscillons and transients during preheating after single field inflation**  |
|| T. Jia, Y. Sang, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-09-09*|
|*Comments*| *14 pages, 12 figures, 2 tables*|
|**Abstract**|            In the single-field model, the preheating process occurs through self-resonance of inflaton field. We study the nonlinear structures generated during preheating in the $\alpha$-attractor models and monodromy models. The potentials have a power law form $\propto\left|\phi\right|^{2n}$ near the origin and a flat region away from bottom, which are consistent with current cosmological observations. The Floquet analysis shows that potential parameters in monodromy model have a significant influence on the region of resonance bands. The analytical oscillons solution for the $\alpha$-attractor T model with parameter $n=1$ is derived using the small amplitude analysis method. Besides we investigate the formation of nonlinear structures, the equation of state and the energy transfer through the (3+1) dimensional lattice simulation. We find that the symmetric T potential and the asymmetric E potential in the $\alpha$-attractor models have similar nonlinear dynamics. And the potential parameter $n$ in monodromy model significantly influences the lifetime of transients, whereas the parameter $q$ exerts minimal impact on the nonlinear dynamics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.04405-b31b1b.svg)](https://arxiv.org/abs/2409.04405) | **Asymptotic-state prediction for fast flavor transformation in neutron star mergers**  |
|| S. Richers, et al. -- incl., <mark>S. Ghosh</mark> |
|*Appeared on*| *2024-09-09*|
|*Comments*| *Comments welcome*|
|**Abstract**|            Neutrino flavor instabilities appear to be omnipresent in dense astrophysical environments, thus presenting a challenge to large-scale simulations of core-collapse supernovae and neutron star mergers (NSMs). Subgrid models offer a path forward, but require an accurate determination of the local outcome of such conversion phenomena. Focusing on "fast" instabilities, related to the existence of a crossing between neutrino and antineutrino angular distributions, we consider a range of analytical mixing schemes, including a new, fully three-dimensional one, and also introduce a new machine learning (ML) model. We compare the accuracy of these models with the results of several thousands of local dynamical calculations of neutrino evolution from the conditions extracted from classical NSM simulations. Our ML model shows good overall performance, but struggles to generalize to conditions from a NSM simulation not used for training. The multidimensional analytic model performs and generalizes even better, while other analytic models (which assume axisymmetric neutrino distributions) do not have reliably high performances, as they notably fail as expected to account for effects resulting from strong anisotropies. The ML and analytic subgrid models extensively tested here are both promising, with different computational requirements and sources of systematic errors.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.03829-b31b1b.svg)](https://arxiv.org/abs/2409.03829) | **RUBIES Reveals a Massive Quiescent Galaxy at z=7.3**  |
|| A. Weibel, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-09-09*|
|*Comments*| *21 pages, 7 figures, submitted to ApJ*|
|**Abstract**|            We report the spectroscopic discovery of a massive quiescent galaxy at $z_{\rm spec}=7.29\pm0.01$, just $\sim700\,$Myr after the Big Bang. RUBIES-UDS-QG-z7 was selected from public JWST/NIRCam and MIRI imaging from the PRIMER survey and observed with JWST/NIRSpec as part of RUBIES. The NIRSpec/PRISM spectrum reveals one of the strongest Balmer breaks observed thus far at $z>6$, no emission lines, but tentative Balmer and Ca absorption features, as well as a Lyman break. Simultaneous modeling of the NIRSpec/PRISM spectrum and NIRCam and MIRI photometry (spanning $0.9-18\,\mu$m) shows that the galaxy formed a stellar mass of log$(M_*/M_\odot)=10.23^{+0.04}_{-0.04}$ in a rapid $\sim 100-200\,$Myr burst of star formation at $z\sim8-9$, and ceased forming stars by $z\sim8$ resulting in $\log \rm{sSFR/yr}^{-1}<-10$. We measure a small physical size of $209_{-24}^{+33}\,{\rm pc}$, which implies a high stellar mass surface density within the effective radius of $\log(\Sigma_{*,\rm e}/{\rm M_\odot\,kpc}^{-2})=10.85_{-0.12}^{+0.11}$ comparable to the densities measured in quiescent galaxies at $z\sim2-5$. The 3D stellar mass density profile of RUBIES-UDS-QG-z7 is remarkably similar to the central densities of local massive ellipticals, suggesting that at least some of their cores may have already been in place at $z>7$. The discovery of RUBIES-UDS-QG-z7 has strong implications for galaxy formation models: the estimated number density of quiescent galaxies at $z\sim7$ is $>100\times$ larger than predicted from any model to date, indicating that quiescent galaxies have formed earlier than previously expected.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 36</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.03816/./FiguresPaper3/2popDustPyCompare_New.png', 'tmp_2409.03816/./FiguresPaper3/FudgeStudy_Diffusion_New.png', 'tmp_2409.03816/./FiguresPaper3/DuffellCompareDetailed_New.png']
copying  tmp_2409.03816/./FiguresPaper3/2popDustPyCompare_New.png to _build/html/
copying  tmp_2409.03816/./FiguresPaper3/FudgeStudy_Diffusion_New.png to _build/html/
copying  tmp_2409.03816/./FiguresPaper3/DuffellCompareDetailed_New.png to _build/html/
exported in  _build/html/2409.03816.md
    + _build/html/tmp_2409.03816/./FiguresPaper3/2popDustPyCompare_New.png
    + _build/html/tmp_2409.03816/./FiguresPaper3/FudgeStudy_Diffusion_New.png
    + _build/html/tmp_2409.03816/./FiguresPaper3/DuffellCompareDetailed_New.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\RN}[1]{$
$  \textup{\uppercase\expandafter{\romannumeral#1}}$
$}$
$\newcommand{\Sigmag}{\Sigma_{\mathrm{g}}}$
$\newcommand{\Sigmad}{\Sigma_{\mathrm{d}}}$
$\newcommand{\rhog}{\rho_{\mathrm{g}}}$
$\newcommand{\rhod}{\rho_{\mathrm{d}}}$
$\newcommand{\vg}{\vec{v}_{\mathrm{g}}}$
$\newcommand{\vd}{\vec{v}_{\mathrm{d}}}$
$\newcommand{\amax}{a_{\mathrm{max}}}$
$\newcommand{\amin}{a_{\mathrm{min}}}$
$\newcommand{\aint}{a_{\mathrm{int}}}$
$\newcommand{\adr}{a_{\mathrm{drift}}}$
$\newcommand{\vdr}{v_{\mathrm{drift}}}$
$\newcommand{\tfr}{t_\mathrm{fric}}$
$\newcommand{\St}{\mathrm{St}}$
$\newcommand{\Rey}{\mathrm{Re}}$
$\newcommand{\OmK}{\Omega_\text{K}}$
$\newcommand{\der}[2]{\frac{\partial{#1}}{\partial{#2}}}$
$\newcommand{\hd}{h_{\mathrm{d}}}$
$\newcommand{\Mp}{m_{\mathrm{p}}}$
$\newcommand{\Sd}{\Sigma_{\mathrm{d}}}$
$\newcommand{\dpy}{\texttt{DustPy}}$
$\newcommand{\pluto}{\texttt{PLUTO}}$
$\newcommand{\tpop}{\texttt{TriPoD}}$
$\newcommand{\tpoppy}{\texttt{two-pop-py}}$
$\newcommand{\rev}[1]{{#1}}$
$\newcommand{\revI}[1]{{#1}}$
$\newcommand{\revII}[1]{{#1}}$
$\newcommand{\new}[1]{{#1}}$
$\newcommand$</div>



<div id="title">

# $\tpop$ : \texttt{Tri}-\texttt{P}opulation size distributions for \texttt{D}ust evolution.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.03816-b31b1b.svg)](https://arxiv.org/abs/2409.03816)<mark>Appeared on: 2024-09-09</mark> -  _22 pages, 13 figures, accepted for publication in Astronomy and Astrophysics_

</div>
<div id="authors">

<mark>T. Pfeil</mark>, T. Birnstiel, <mark>H. Klahr</mark>

</div>
<div id="abstract">

**Abstract:** Dust coagulation and fragmentation impact the structure and evolution of protoplanetary disks and set the initial conditions for planet formation. Dust grains dominate the opacities, they determine the cooling times of the gas via thermal accommodation in collisions, they influence the ionization state of the gas, and the available grain surface area is an important parameter for the chemistry in protoplanetary disks. Therefore, dust evolution is an effect that should not be ignored in numerical studies of protoplanetary disks. Available dust coagulation models are, however, too computationally expensive to be implemented in large-scale hydrodynamic simulations. This limits detailed numerical studies of protoplanetary disks, including these effects, mostly to one-dimensional models. We aim to develop a simple---yet accurate---dust coagulation model that can be easily implemented in hydrodynamic simulations of protoplanetary disks. Our model shall not significantly increase the computational cost of simulations and provide information about the local grain size distribution. The local dust size distributions are assumed to be truncated power laws. Such distributions can be fully characterized by only two dust fluids (large and small grains) and a maximum particle size, truncating the power law. We compare our model to state-of-the-art dust coagulation simulations and calibrate it to achieve a good fit with these sophisticated numerical methods. Running various parameter studies, we achieved a good fit between our simplified three-parameter model and $\dpy$ , a state-of-the-art dust coagulation software. We present $\tpop$ , a sub-grid dust coagulation model for the $\pluto$ code. With $\tpop$ , we can perform two-dimensional, vertically integrated dust coagulation simulations on top of a hydrodynamic simulation. Studying the dust distributions in two-dimensional vortices and planet-disk systems is thus made possible.

</div>

<div id="div_fig1">

<img src="tmp_2409.03816/./FiguresPaper3/2popDustPyCompare_New.png" alt="Fig3" width="100%"/>

**Figure 3. -** Comparison between the fiducial $\dpy$  setup and the fiducial $\tpop$  setup.Comparison between the full coagulation model $\dpy$(upper row, 171 dust fluids) and our new three-parameter power-law prescription $\tpop$ , which we implemented in the $\pluto$  code (two dust fluids). We show two snapshots of the one-dimensional disk models in the first two panels in each row. The third panel shows the local dust size distribution of the respective model at \SI{10}{\AU} for both snapshots. The light grid in the background represents size distribution power laws with $n(a)\propto a^{-3.5}$ and $n(a)\propto a^{-3.0}$. (*fig:fiducial*)

</div>
<div id="div_fig2">

<img src="tmp_2409.03816/./FiguresPaper3/FudgeStudy_Diffusion_New.png" alt="Fig6" width="100%"/>

**Figure 6. -** Drift velocity factor parameter study.Comparison between $\dpy$  and our model in a setup with dust diffusion ($\delta=10^{-3}$) and with different drift calibration factors $f_\mathrm{drift}$. $\rev$I{Solid lines show the results of our $\tpop$  calibration runs and dashed lines show the respective $\dpy$  simulation, to which we calibrate our model.} The upper row shows a timeseries of the dust column density evolution in three snapshots. In the lower row, we show the mass evolution and the errors with respect to the full coagulation model $\dpy$ . For a factor of $\new${$f_\mathrm{drift}=0.8$}, the mass evolution of the full coagulation model is well reproduced by our three-parameter model. (*fig:FudgeDiff*)

</div>
<div id="div_fig3">

<img src="tmp_2409.03816/./FiguresPaper3/DuffellCompareDetailed_New.png" alt="Fig9" width="100%"/>

**Figure 9. -** Detailed comparison between $\dpy$  and $\tpop$  in a simulation with a planetary gap.Comparison between a $\dpy$  simulation and a $\pluto$  simulation with our $\tpop$  dust coagulation model in a protoplanetary disk with a pre-defined gap. We also show the particle sizes and dust column density profiles calculated with the old $\tpop$py  model as dotted lines. $\rev$I{The plot on the right-hand side depicts the local dust size distributions at \SI{5}{\AU} (within the gap) and at \SI{8.5}{\AU} (at the pressure maximum), for two different snapshots. The gray lines in the right panel follow power laws with $q=-3.5$ and $q=-3.0$.} (*fig:Duffell1D*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.03816"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

176  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
